In [1]:
from dotenv import load_dotenv
# Importing the keys
load_dotenv()
import os
import requests
import json

# Download the data

In [2]:
import requests

url = "https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json"
response = requests.get(url)

if response.status_code == 200:
    with open("documents.json", "wb") as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to download the file.")

File downloaded successfully.


## Loading the Data

In [2]:
with open("documents.json","rt") as f:
    file =json.load(f)

documents = []

for course in file:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [1]:
#documents

In [5]:
course_name

'mlops-zoomcamp'

In [4]:
from elasticsearch import Elasticsearch


In [5]:
es_client = Elasticsearch('http://localhost:9200') 


In [6]:
es_client.info()

ObjectApiResponse({'name': '3d1233600939', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'j6oUujGNSja-CYtFdVBopw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [8]:
from tqdm.auto import tqdm


In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [11]:
query = 'I just disovered the course. Can I still join it?'


In [12]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                #"filter": {
                 #   "term": {
                  #      "course": "data-engineering-zoomcamp"
                  #  }
                #}
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [15]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [20]:
from openai import OpenAI
client =OpenAI()

In [18]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [13]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [21]:
rag(query)

"Yes, you can still join the course even if you discovered it after the start date. You don't need to register to be eligible to submit the homework. However, keep in mind that there are deadlines for turning in the final projects, so it's important not to leave everything until the last minute."